<a href="https://colab.research.google.com/github/GretaDanesi18/Olive_Detection_Tracking/blob/main/YOLOv5_olive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#clono la repo e installa requisiti

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
! pip install -r requirements.txt

In [ ]:
#librerie

In [ ]:
import torch 
import os
import random 
import shutil
from IPython.display import Image
from PIL import Image, ImageDraw
import cv2
import pandas as pd
import subprocess
from numpy import source


In [ ]:
#controllo utilizzo del device GPU oppure CPU

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.11.0+cu113  Device: cuda


In [ ]:
#clono repo con dataset e file yaml

In [ ]:
!git clone https://github.com/GretaDanesi18/Olive_Detection_Tracking.git

Cloning into 'Olive_Detection_Tracking'...
remote: Enumerating objects: 500, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 500 (delta 1), reused 8 (delta 1), pack-reused 492
Receiving objects: 100% (500/500), 227.17 MiB | 29.58 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
#creo le directory per fare split del dataset

In [ ]:
try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/images/val'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/images/val')

except OSError:
    print('Errore nella creazione della cartella images/val')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/images/train'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/images/train')

except OSError:
    print('Errore nella creazione della cartella images/train')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/labels/val'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/labels/val')

except OSError:
    print('Errore nella creazione della cartella labels/val')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/labels/train'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/labels/train')

except OSError:
    print('Errore nella creazione della cartella labels/train')


In [ ]:
#array di appoggio per i dati
#percentuali per lo split

In [ ]:
#array che contiene i file
image = []
label = [] 

#path
all_data_path = '/content/yolov5/Olive_Detection_Tracking/dataset'
train_image_path = '/content/yolov5/Olive_Detection_Tracking/images/train'
val_image_path = '/content/yolov5/Olive_Detection_Tracking/images/val'
train_label_path ='/content/yolov5/Olive_Detection_Tracking/labels/train'
val_label_path = '/content/yolov5/Olive_Detection_Tracking/labels/val'


#percentuale per train, val
val_ratio = 0.2
train_ratio = 0.8

#inserisco file negli array corrispondenti
for (dirname, dirs, files) in os.walk(all_data_path):
    for filename in files:
        if filename.endswith('.txt'):
            label.append(filename)
        else:
            image.append(filename)

#numero di immagini per train e per val
num_train = int(len(image)*train_ratio)
num_val = int(len(image)*val_ratio)

print("Immagini per il training: ", num_train)
print("Immagini per validation: ", num_val)
print(image[3])
print(label[3])


In [ ]:
#inserisco dati nelle directory per yolov5

In [ ]:
#inserisco image e label in train
for i in range(num_train):
   file_image = random.choice(image)
   file_image_name = file_image.split('.')[0]
   file_label =  file_image_name+'.txt'

   #sposto i file nella cartella train
   shutil.copy(os.path.join(all_data_path, file_image), os.path.join(train_image_path, file_image))
   shutil.copy(os.path.join(all_data_path, file_label), os.path.join(train_label_path, file_label))

   #rimuovo i file da arrays
   image.remove(file_image)
   label.remove(file_label)
    

#inserisco image e label in val  
for i in range(num_val):
    file_image =random.choice(image) # get name of random image from origin dir
    file_image_name = file_image.split('.')[0]
    file_label = file_image_name+'.txt'

    #move both files into train dir
    shutil.copy(os.path.join(all_data_path, file_image), os.path.join(val_image_path, file_image))
    shutil.copy(os.path.join(all_data_path, file_label), os.path.join(val_label_path, file_label))
    
    #rimuovo i file da arrays
    image.remove(file_image)
    label.remove(file_label)


In [ ]:
#inserisco file yaml in directory data

In [ ]:
source = 'Olive_Detection_Tracking/dataset.yaml'
destination = 'data/dataset.yaml'
shutil.copy(source,destination)

'data/dataset.yaml'

In [ ]:
#Weights & Biases
%pip install -q wandb
import wandb
wandb.login()

     |████████████████████████████████| 1.8 MB 7.8 MB/s 
     |████████████████████████████████| 181 kB 74.6 MB/s 
     |████████████████████████████████| 145 kB 65.3 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#addestramento

In [ ]:
!python train.py --img 416 --batch 16 --epochs 50 --data dataset.yaml --weights yolov5s.pt --cache 

wandb: Currently logged in as: danesig. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=, data=dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-289-g526e650 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, o

In [ ]:
#visualizzazione dati 

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
#Controllo dove vengono salvati runs/train/.. i risulati e aggiorna directory

In [ ]:
Image.open('runs/train/exp/train_batch0.jpg')

In [ ]:
Image.open('runs/train/exp/val_batch0_labels.jpg')

In [ ]:
Image.open('runs/train/exp/val_batch0_pred.jpg')

In [ ]:

file_csv=pd.read_csv('runs/train/exp/results.csv')
file_csv.head(-1)


In [ ]:
Image.open('runs/train/exp/results.png')

In [ ]:
!python detect.py --source Olive_Detection_Tracking/frame_video  --weights runs/train/exp/weights/best.pt --save-txt

In [ ]:
Image.open('runs/detect/exp3/frame_2_9.jpg')